In [1]:
import numpy as np
import string 
import scipy.io
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
import itertools
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPRegressor
import numpy as np
import string 
import scipy.io
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
import itertools
from sklearn import svm
import statsmodels.api as sm # import statsmodels 
from scipy import signal
from sklearn.decomposition import *
from sklearn.linear_model import *
import sklearn
from sklearn.linear_model import Ridge
from sklearn.metrics import explained_variance_score

class Scan(object):
    def __init__(self,activations,timestamp, step,prev_words=None,next_words=None,all_words=None,all_pos=None,all_speak_features=None,current_translated_words=None):
        self.activations = activations
        self.timestamp = timestamp
        self.prev_words = prev_words
        self.next_words = next_words
        self.step = step
        self.all_words = all_words
        self.current_translated_words = current_translated_words
        self.all_pos = all_pos
        self.all_speak_features = all_speak_features
        self.brain3d = None
        
        
        
subject_id = 1

In [24]:
def eval(dists,e_dists):
    nn_index = np.argmin(dists,axis=1)
    accuracy_on_test = np.mean(nn_index == np.argmax(np.eye(dists.shape[0]),axis=1))


    b_acc = []
    e_b_acc = []
    for i,j in itertools.combinations(np.arange(dists.shape[0]), 2):
        right_match = dists[i,i] + dists[j,j]
        wrong_match = dists[i,j] + dists[j,i]
        b_acc.append(right_match < wrong_match)

        e_right_match = e_dists[i,i] + e_dists[j,j]
        e_wrong_match = e_dists[i,j] + e_dists[j,i]
        e_b_acc.append(e_right_match < e_wrong_match)

    #print("binary accuracy: ", np.mean(b_acc)," ", np.mean(e_b_acc))
    return np.mean(b_acc),np.mean(e_b_acc),b_acc,e_b_acc

def train_model_regression(X,y):
    #X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model
    # Note the difference in argument order
    #model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
    
    #model = sklearn.linear_model.LinearRegression( normalize=True).fit(X,y)
    model = Ridge(alpha=1.0).fit(X,y)
    return model


def MRR(distances):
    prec_at_corrects = []
    ranks = []
    sorted_indexes = np.argsort(distances,axis=1)
    for i in np.arange(len(distances)):
        #print(i)
        correct_at = np.where(sorted_indexes[i] == i)[0] + 1
        #print("Reciprocal Rank",correct_at)
        prec_at_correct = 1.0/correct_at
        #print("precision at ",correct_at,": ",prec_at_correct)
        prec_at_corrects.append(prec_at_correct)
        ranks.append(correct_at)
    
    print("MRR: ",np.mean(prec_at_corrects)," ",np.mean(ranks))
    return np.mean(ranks), np.mean(prec_at_corrects), ranks,prec_at_corrects

def test_model(model,X_t,y_t):
    #X_t = sm.add_constant(X_t) ## let's add an intercept (beta_0) to our model
    pred_t = model.predict(X_t)
    
    
    argmax_accuracy = np.mean(np.argmax(pred_t, axis=1) == np.argmax(y_t,axis=1))
    print("check argmax dim:",np.argmax(pred_t, axis=1).shape)
    print("argmax accuracy:",argmax_accuracy)
    cosine_dists = distance.cdist(pred_t,y_t,'cosine')
    euc_dists =  distance.cdist(pred_t,y_t,'euclidean')
    
    print("cosine dist >>")
    mean_ranks_c, mean_mrp_c, ranks,prec_at_corrects = MRR(cosine_dists)
    
    print("euc_dists dist >>")
    mean_ranks_e = MRR(euc_dists)
    
    print("binary accuracy >>")
    c_acc, e_acc, _,_ = eval(cosine_dists,euc_dists)
    print(c_acc,e_acc)
    exp_var = explained_variance_score(y_t, pred_t, multioutput='uniform_average')
    print("explained variance: ",exp_var)
    return c_acc, e_acc, mean_mrp_c

    


In [37]:
def fold_brain_features(block_features, block_scans, block_ids):
    brains = []
    features = []
    for block_id in block_ids:
        for scan in block_scans[block_id]:
            block_step = scan.step  - block_scans[block_id][0].step
            brains.append(scan.activations[0])

            related_features = []
            i = 0
            while i<4 and (block_step + i) < len(block_features[block_id]):
                related_features.append(block_features[block_id][block_step + i])
                i += 1

            features.append(max(related_features))  
    
    print(len(features),len(brains))
    brains = np.asarray(brains)
    brains = (brains - np.min(brains,axis=0))/(np.max(brains,axis=0) - np.min(brains,axis=0) + 0.000001)
    return brains, features     


def evaluate_fold_brain2features_classifier(feature_model, block_scans,test_blocks,train_blocks):
    brains_train, features_train = fold_brain_features(feature_model,block_scans,block_ids=train_blocks)
    brains_test, features_test = fold_brain_features(feature_model,block_scans,block_ids=test_blocks)


    clf = SVC(kernel='linear',class_weight='balanced', C=0.1, probability=True)
    clf.fit(brains_train, features_train)

    prediction_train = clf.predict(brains_train)
    prediction_test = clf.predict(brains_test)

    train_accuracy = np.mean(features_train == prediction_train)
    test_accuracy = np.mean(features_test == prediction_test)

    print("train accuracy: ",train_accuracy, " test accuracy: ",test_accuracy)
    cnf_matrix = confusion_matrix(features_test, prediction_test)
    print(cnf_matrix)
    
    
    result = []
    accu_result = []
    for i in np.arange(len(brains_test)):
        if features_test[i] > 0:
            j = np.where (np.argsort(prediction_test[i]) == np.asarray(features_test[i]))
            if len(j[0]) > 1:
                result.append(1/((np.max(features_test)+1) - j[0][0]))
                accu_result.append(j[0][0] == 0)
            else:
                result.append(1/((np.max(features_test)+1) - j[0]))
                accu_result.append(j[0] == 0)
    print("MRR: ",np.mean(result), "important accuracy:",np.mean(accu_result))
    
def fold_brain_features(block_features, block_scans, block_ids):
    brains = []
    features = []
    for block_id in block_ids:
        for scan in block_scans[block_id]:
            block_step = scan.step  - block_scans[block_id][0].step
            brains.append(scan.activations[0])

            related_features = []
            i = 0
            while i<4 and (block_step + i) < len(block_features[block_id]):
                related_features.append(block_features[block_id][block_step + i])
                i += 1

            features.append(max(related_features))  
    
    print(len(features),len(brains))
    brains = np.asarray(brains)
    brains = (brains - np.min(brains,axis=0))/(np.max(brains,axis=0) - np.min(brains,axis=0) + 0.000001)
    return brains, features     


def evaluate_fold_brain2features_classifier(feature_model, block_scans,test_blocks,train_blocks):
    brains_train, features_train = fold_brain_features(feature_model,block_scans,block_ids=train_blocks)
    brains_test, features_test = fold_brain_features(feature_model,block_scans,block_ids=test_blocks)


    clf = SVC(kernel='linear',class_weight='balanced', C=0.1, probability=True)
    clf.fit(brains_train, features_train)

    prediction_train = clf.predict(brains_train)
    prediction_test = clf.predict(brains_test)

    train_accuracy = np.mean(features_train == prediction_train)
    test_accuracy = np.mean(features_test == prediction_test)

    print("train accuracy: ",train_accuracy, " test accuracy: ",test_accuracy)
    cnf_matrix = confusion_matrix(features_test, prediction_test)
    print(cnf_matrix)
    
    
    result = []
    accu_result = []
    for i in np.arange(len(brains_test)):
        if features_test[i] > 0:
            j = np.where (np.argsort(prediction_test[i]) == np.asarray(features_test[i]))
            if len(j[0]) > 1:
                result.append(1/((np.max(features_test)+1) - j[0][0]))
                accu_result.append(j[0][0] == 0)
            else:
                result.append(1/((np.max(features_test)+1) - j[0]))
                accu_result.append(j[0] == 0)
    print("MRR: ",np.mean(result), "important accuracy:",np.mean(accu_result))

In [4]:
reduced_brain_vectors = np.load("../processed_data/512reducted_brain_scans.npy")
not_reduced_brain_vectors = np.load("../processed_data/reducted_brain_scans.npy")
reduced_brain_vectors_words = np.load("../processed_data/reducted_words.npy")

story_features = scipy.io.loadmat('../data/story_features.mat') 
subject_file = scipy.io.loadmat('../data/'+'subject_'+str(subject_id)+'.mat') 
part_of_speaches_feature_id = 8
speach_feature_id = 1
part_of_speaches = story_features['features'][0][part_of_speaches_feature_id][1][0]
part_of_speaches_features = story_features['features'][0][part_of_speaches_feature_id][2]
speaches = story_features['features'][0][speach_feature_id][1][0]
speach_features = story_features['features'][0][speach_feature_id][2]


speach_feature_id = 1
motion_feature_id = 2
emotion_feature_id = 3
verbs_feature_id = 4
characters_feature_id = 5
visual_wordlength_feature_id = 6
Word_Num_feature_id = 7
part_of_speaches_feature_id = 8
Dependency_role_feature_id = 9

word_index = 0
time_index = 1

In [6]:
actual_words = []
word_times = []
word_pos = []
speach_states = []
for i in np.arange(subject_file['words'].shape[1]):
    actual_words.append(subject_file['words'][0][i][word_index][0][0][0].strip().replace("@",""))
    word_times.append(subject_file['words'][0][i][time_index][0][0])
    word_pos.append(part_of_speaches_features[i])
    speach_states.append(speach_features[i])
    
    
blocks = subject_file['time'][:,1]
scan_times = subject_file['time'][:,0]

block_ends = {}
block_ends_indexes = {}


for block_id in np.arange(1,5):
    block_ends_indexes[block_id] = np.where(scan_times == np.max(scan_times[np.where(blocks == block_id )]))[0]
    block_ends[block_id] = scan_times[block_ends_indexes[block_id]]+2


print(block_ends)
print(block_ends_indexes)

block_texts = {1:[],2:[],3:[],4:[]}
block_pos = {1:[],2:[],3:[],4:[]}
block_speach_state = {1:[],2:[],3:[],4:[]}
block_steps = {1:[],2:[],3:[],4:[]}
character_feature = {1:[],2:[],3:[],4:[]}
block_id = 1
for index in np.arange(len(actual_words)):
    if word_times[index] > block_ends[block_id]:
        block_id += 1
    block_texts[block_id].append(str(actual_words[index].encode("ascii",'ignore').decode()))
    block_pos[block_id].append(word_pos[index])
    block_speach_state[block_id].append(speach_states[index][0])
    block_steps[block_id].append(index)
    character_feature[block_id].append(story_features['features'][0][characters_feature_id][2][index])

{1: array([680], dtype=uint16), 2: array([1384], dtype=uint16), 3: array([1942], dtype=uint16), 4: array([2702], dtype=uint16)}
{1: array([339]), 2: array([691]), 3: array([970]), 4: array([1350])}


In [7]:
import re
import string

end_of_sentence_indexes = {1:[],2:[],3:[],4:[]}
start_of_sentence_indexes = {1:[],2:[],3:[],4:[]}
qout_indexes = {1:[],2:[],3:[],4:[]}
inside_qout_indexes = {1:[],2:[],3:[],4:[]}

for block_id in [1,2,3,4]:
    start_of_sentence_indexes[block_id].append(0)
    inside_qout = False
    for i in np.arange(len(block_texts[block_id])):
        
        already_in = False   
        if "\"" in block_texts[block_id][i]:
            qout_indexes[block_id].append(i)
            
            checked = False
            if block_texts[block_id][i].strip().startswith("\""):
                if inside_qout is False :
                    if already_in == False:
                        inside_qout_indexes[block_id].append(i)
                        already_in = True
                inside_qout = not inside_qout
                checked = True
                
            if block_texts[block_id][i].strip().endswith("\""):
                if inside_qout is True :
                    if already_in == False:
                        inside_qout_indexes[block_id].append(i) 
                        already_in = True
                inside_qout = not inside_qout
                checked = True
                
            if checked == False:
                print(block_texts[block_id][i])
                
        if inside_qout:
            if already_in == False:
                inside_qout_indexes[block_id].append(i)
        #end/start of sentence
        truth_table = [punc in block_texts[block_id][i] for punc in ["!",".","?",":"]]
        if True in truth_table:
            end_of_sentence_indexes[block_id].append(i)
            if i+1 < len(block_texts[block_id]):
                start_of_sentence_indexes[block_id].append(i+1)
        

In [8]:
inside_qout_feature = {1:[],2:[],3:[],4:[]}
start_sentence_feature = {1:[],2:[],3:[],4:[]}
end_sentence_feature = {1:[],2:[],3:[],4:[]}
for block_id in [1,2,3,4]:
    inside_qout_feature[block_id] = np.zeros((len(block_texts[block_id])))
    inside_qout_feature[block_id][inside_qout_indexes[block_id]] = 1
    
    start_sentence_feature[block_id] = np.zeros((len(block_texts[block_id])))
    start_sentence_feature[block_id][start_of_sentence_indexes[block_id]] = 1

    end_sentence_feature[block_id] = np.zeros((len(block_texts[block_id])))
    end_sentence_feature[block_id][end_of_sentence_indexes[block_id]] = 1
    

In [9]:
def get_brain_mapped_feature(block_feature):
    brain_mapped_features = []
    for block_id in [1,2,3,4]:
        for j in np.arange(0,len(block_feature[block_id]),4):
            i = 0
            related_features = []
            while i < 4 and (i+j) < len(block_feature[block_id]):
                if max(block_feature[block_id][i+j]) == 0:
                    related_features.append(0)
                else:
                    related_features.append(1+ np.argmax(block_feature[block_id][i+j]))
                i += 1

            brain_mapped_features.append(np.max(related_features))   


In [10]:
LSTM_embeddings_H_0 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_H_1 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_H_01 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_M_0 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_M_1 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_M_01 = {1:[],2:[],3:[],4:[]}

for block_id in [1,2,3,4]:    
    print("block id",block_id)
    layer_id = 0
    lstm_h_0 = np.load("../../lm1b/text_input_full_state/block_"+str(block_id)+"/lstm_hidden_"+str(layer_id)+".npy")
    lstm_c_0 = np.load("../../lm1b/text_input_full_state/block_"+str(block_id)+"/lstm_memory_"+str(layer_id)+".npy")
    layer_id = 1
    lstm_h_1 = np.load("../../lm1b/text_input_full_state/block_"+str(block_id)+"/lstm_hidden_"+str(layer_id)+".npy")
    lstm_c_1 = np.load("../../lm1b/text_input_full_state/block_"+str(block_id)+"/lstm_memory_"+str(layer_id)+".npy")
  
    for step in np.arange(len(block_texts[block_id])):
        LSTM_embeddings_H_0[block_id].append(lstm_h_0.item()[step][0])
        LSTM_embeddings_H_1[block_id].append(lstm_h_1.item()[step][0])
        LSTM_embeddings_H_01[block_id].append(np.concatenate([lstm_h_0.item()[step][0],lstm_h_1.item()[step][0]]))
        
        LSTM_embeddings_M_0[block_id].append(lstm_c_0.item()[step][0])
        LSTM_embeddings_M_1[block_id].append(lstm_c_1.item()[step][0])
        LSTM_embeddings_M_01[block_id].append(np.concatenate([lstm_c_0.item()[step][0],lstm_c_1.item()[step][0]]))

block id 1
block id 2
block id 3
block id 4


In [10]:
print(len(inside_qout_feature[1]))
print(len(start_sentence_feature[1]))
print(len(end_sentence_feature[1]))
print(len(block_pos[1]))
print(len(block_speach_state[1]))
print(len(character_feature[1]))
print(len(LSTM_embeddings_H_0[1]))
print(len(block_steps[1]))

1303
1303
1303
1303
1303
1303
1303
1303


In [11]:
subject_id = 1
fMRI_preprocessing_mode = "detrend_filter_std" 
block_scans = np.load("../processed_data/"+"subject_"+str(subject_id)+"/"+fMRI_preprocessing_mode+"subject_"+str(subject_id)+"_scan_objects.npy").item()

In [12]:
print(len(block_scans[1])*4)

1304


In [13]:
def prepare_data(train_block_ids,test_block_ids,block_scans,block_features, mode="concat"):
    train_brain_activations = []
    
    train_features = {}
    for feautre_key in block_features.keys(): 
        train_features[feautre_key] = []
        
    for train_block_id in train_block_ids:
        for i in np.arange(len(block_scans[train_block_id])):
            scan = block_scans[train_block_id][i]
            train_brain_activations.append(scan.activations[0])
        
            step = scan.step - block_scans[train_block_id][0].step

            for feautre_key in block_features.keys(): 
                i = 0
                feature_values = []
                
                while i< 4 and (step+i) < len(block_features[feautre_key][train_block_id]):
                    feature_values.append(block_features[feautre_key][train_block_id][step+i])
                    i += 1
                while  len(feature_values) < 4:
                    feature_values.append(np.zeros_like(feature_values[-1]))
                

                if mode == "concat":
                    if len(np.asarray(feature_values).shape) > 1:
                        train_features[feautre_key].append(np.concatenate(np.asarray(feature_values)))
                    else:
                        train_features[feautre_key].append(np.asarray(feature_values))

                else:
                    train_features[feautre_key].append(np.mean(feature_values,axis=0))
                    
       
    test_brain_activations = []
    test_features = {}
    for feautre_key in block_features.keys(): 
        test_features[feautre_key] = []
        
    for test_block_id in test_block_ids:
        for i in np.arange(len(block_scans[test_block_id])):
            scan = block_scans[test_block_id][i]
            test_brain_activations.append(scan.activations[0])
        
            step = scan.step - block_scans[test_block_id][0].step
            for feautre_key in block_features.keys(): 
                i = 0
                feature_values = []
                while i< 4 and (step+i) < len(block_features[feautre_key][test_block_id]):
                    feature_values.append(block_features[feautre_key][test_block_id][step+i])
                    i += 1

                while  len(feature_values) < 4:
                    feature_values.append(np.zeros_like(feature_values[-1]))
                    
                if mode == "concat":
                    if len(np.asarray(feature_values).shape) > 1:
                        test_features[feautre_key].append(np.concatenate(feature_values))
                    else:
                        test_features[feautre_key].append(feature_values)
                else:
                    test_features[feautre_key].append(np.mean(feature_values,axis=0))

    
    return train_features,train_brain_activations,test_features,test_brain_activations





In [14]:
print(len(inside_qout_feature[1]))
print(len(start_sentence_feature[1]))
print(len(end_sentence_feature[1]))
print(len(block_pos[1]))
print(len(block_speach_state[1]))
print(len(character_feature[1]))
print(len(LSTM_embeddings_H_0[1]))

block_features = {'inside_qout_feature':inside_qout_feature,
                  'start_sentence_feature':start_sentence_feature,
                  'end_sentence_feature':end_sentence_feature,
                 'pos_tag':block_pos,
                 'speach_state':block_speach_state,
                 'character':character_feature,
                 'LSTM_H_0':LSTM_embeddings_H_0,
                 'LSTM_H_1':LSTM_embeddings_H_1,
                 'LSTM_M_0':LSTM_embeddings_M_0,
                 'LSTM_M_1':LSTM_embeddings_M_1,
                 'LSTM_H_01':LSTM_embeddings_H_01,
                 'LSTM_M_01':LSTM_embeddings_M_01,
                 'step':block_steps}

1303
1303
1303
1303
1303
1303
1303


In [15]:
train_features,train_brain_activations,test_features,test_brain_activations = {},{},{},{}
train_features[1],train_brain_activations[1],test_features[1],test_brain_activations[1] = \
prepare_data([1,2,3],[4],block_scans,block_features, mode="concat")

train_features[2],train_brain_activations[2],test_features[2],test_brain_activations[2] = \
prepare_data([2,3,4],[1],block_scans,block_features, mode="concat")

train_features[3],train_brain_activations[3],test_features[3],test_brain_activations[3] = \
prepare_data([1,3,4],[2],block_scans,block_features, mode="concat")

train_features[4],train_brain_activations[4],test_features[4],test_brain_activations[4] = \
prepare_data([1,2,4],[3],block_scans,block_features, mode="concat")


MRR_score = {}
Accuracy_score = {}

In [36]:
print(np.asarray(train_features[1]['LSTM_H_01']).shape)
X =  train_features[1]['LSTM_H_01']## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations[1]) ## Y usually means our output/dependent variable

X_t = test_features[1]['LSTM_H_01'] ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations[1]) ## Y usually means our output/dependent variable

#X = (X - np.min(X,axis=0))/ ( np.max(X,axis=0) - np.min(X,axis=0)+ 0.00000001)
#X_t = (X_t - np.min(X_t,axis=0))/ ( np.max(X_t,axis=0) - np.min(X_t,axis=0)+ 0.000000001)

pca_x = PCA(n_components=512)
pca_x.fit(X)

X = pca_x.transform(X)
X_t = pca_x.transform(X_t)

#pca_y = PCA(n_components=2048)
#pca_y.fit(y)

#y = pca_y.transform(y)
#y_t = pca_y.transform(y_t)

#X = X - np.mean(X,axis=0)
#X_t = X_t - np.mean(X_t, axis=0)
#y = y - np.mean(y,axis=0)
#y_t = y_t - np.mean(y_t, axis=0)

y = (y - np.min(y,axis=0))/ ( np.max(y,axis=0) - np.min(y,axis=0)+ 0.0000001)
y_t = (y_t - np.min(y_t,axis=0))/ ( np.max(y_t,axis=0) - np.min(y_t,axis=0)+ 0.0000001)



model = Ridge(alpha=10.0,normalize=True).fit(X,y)
test_model(model,X_t,y_t)


(929, 8192)
check argmax dim: (366,)
argmax accuracy: 0.0
cosine dist >>
MRR:  0.01740836510498491   183.19398907103826
euc_dists dist >>
MRR:  0.017808499638009852   183.26502732240436
binary accuracy >>
0.5433640242533124 0.5311026274421738
explained variance:  0.004761676740323697


(0.5433640242533124, 0.5311026274421738, 0.01740836510498491)

In [ ]:
print(np.asarray(train_features['LSTM_M_1']).shape)
X =  train_features['LSTM_M_1']## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable

X_t = test_features['LSTM_M_1'] ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable


#X = X - np.mean(X,axis=0)
#X_t = X_t - np.mean(X_t, axis=0)
#y = y - np.mean(y,axis=0)
#y_t = y_t - np.mean(y_t, axis=0)

#y = (y - np.min(y,axis=0))/ ( np.max(y,axis=0) - np.min(y,axis=0)+ 0.0000001)
#y_t = (y_t - np.min(y_t,axis=0))/ ( np.max(y_t,axis=0) - np.min(y_t,axis=0)+ 0.0000001)

#X = (X - np.min(X,axis=0))/ ( np.max(X,axis=0) - np.min(X,axis=0)+ 0.00000001)
#X_t = (X_t - np.min(X_t,axis=0))/ ( np.max(X_t,axis=0) - np.min(X_t,axis=0)+ 0.000000001)

pca_x = PCA(n_components=512)
pca_x.fit(X)

X = pca_x.transform(X)
X_t = pca_x.transform(X_t)

pca_y = PCA(n_components=2048)
pca_y.fit(y)

y = pca_y.transform(y)
y_t = pca_y.transform(y_t)


model = train_model(X,y)
test_model(model,X_t,y_t)


In [ ]:
print(np.asarray(train_features['LSTM_M_0']).shape)
X =  train_features['LSTM_M_0']## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable

X_t = test_features['LSTM_M_0'] ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable


#X = X - np.mean(X,axis=0)
#X_t = X_t - np.mean(X_t, axis=0)
#y = y - np.mean(y,axis=0)
#y_t = y_t - np.mean(y_t, axis=0)

#y = (y - np.min(y,axis=0))/ ( np.max(y,axis=0) - np.min(y,axis=0)+ 0.0000001)
#y_t = (y_t - np.min(y_t,axis=0))/ ( np.max(y_t,axis=0) - np.min(y_t,axis=0)+ 0.0000001)

#X = (X - np.min(X,axis=0))/ ( np.max(X,axis=0) - np.min(X,axis=0)+ 0.00000001)
#X_t = (X_t - np.min(X_t,axis=0))/ ( np.max(X_t,axis=0) - np.min(X_t,axis=0)+ 0.000000001)

pca_x = PCA(n_components=512)
pca_x.fit(X)

X = pca_x.transform(X)
X_t = pca_x.transform(X_t)

pca_y = PCA(n_components=2048)
pca_y.fit(y)

y = pca_y.transform(y)
y_t = pca_y.transform(y_t)

model = train_model(X,y)
test_model(model,X_t,y_t)


In [ ]:
print(np.asarray(train_features['LSTM_H_0']).shape)
X =  train_features['LSTM_H_0']## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable

X_t = test_features['LSTM_H_0'] ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable


#X = X - np.mean(X,axis=0)
#X_t = X_t - np.mean(X_t, axis=0)
#y = y - np.mean(y,axis=0)
#y_t = y_t - np.mean(y_t, axis=0)

#y = (y - np.min(y,axis=0))/ ( np.max(y,axis=0) - np.min(y,axis=0)+ 0.0000001)
#y_t = (y_t - np.min(y_t,axis=0))/ ( np.max(y_t,axis=0) - np.min(y_t,axis=0)+ 0.0000001)

#X = (X - np.min(X,axis=0))/ ( np.max(X,axis=0) - np.min(X,axis=0)+ 0.00000001)
#X_t = (X_t - np.min(X_t,axis=0))/ ( np.max(X_t,axis=0) - np.min(X_t,axis=0)+ 0.000000001)

pca_x = PCA(n_components=512)
pca_x.fit(X)

X = pca_x.transform(X)
X_t = pca_x.transform(X_t)

pca_y = PCA(n_components=2048)
pca_y.fit(y)

y = pca_y.transform(y)
y_t = pca_y.transform(y_t)

model = train_model(X,y)
test_model(model,X_t,y_t)


In [ ]:
print(np.asarray(train_features['LSTM_H_01']).shape)
X =  train_features['LSTM_H_01']## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable

X_t = test_features['LSTM_H_01'] ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable


#X = X - np.mean(X,axis=0)
#X_t = X_t - np.mean(X_t, axis=0)
#y = y - np.mean(y,axis=0)
#y_t = y_t - np.mean(y_t, axis=0)

#y = (y - np.min(y,axis=0))/ ( np.max(y,axis=0) - np.min(y,axis=0)+ 0.0000001)
#y_t = (y_t - np.min(y_t,axis=0))/ ( np.max(y_t,axis=0) - np.min(y_t,axis=0)+ 0.0000001)

#X = (X - np.min(X,axis=0))/ ( np.max(X,axis=0) - np.min(X,axis=0)+ 0.00000001)
#X_t = (X_t - np.min(X_t,axis=0))/ ( np.max(X_t,axis=0) - np.min(X_t,axis=0)+ 0.000000001)

pca_x = PCA(n_components=512)
pca_x.fit(X)

X = pca_x.transform(X)
X_t = pca_x.transform(X_t)

pca_y = PCA(n_components=2048)
pca_y.fit(y)

y = pca_y.transform(y)
y_t = pca_y.transform(y_t)

model = train_model(X,y)
test_model(model,X_t,y_t)


In [ ]:

X =  np.concatenate([train_features['LSTM_H_01'],train_features['LSTM_M_01']],axis=1)## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable

X_t = np.concatenate([test_features['LSTM_H_01'],test_features['LSTM_M_01']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable

print(np.asarray(X).shape)

#X = X - np.mean(X,axis=0)
#X_t = X_t - np.mean(X_t, axis=0)
#y = y - np.mean(y,axis=0)
#y_t = y_t - np.mean(y_t, axis=0)

#y = (y - np.min(y,axis=0))/ ( np.max(y,axis=0) - np.min(y,axis=0)+ 0.0000001)
#y_t = (y_t - np.min(y_t,axis=0))/ ( np.max(y_t,axis=0) - np.min(y_t,axis=0)+ 0.0000001)

#X = (X - np.min(X,axis=0))/ ( np.max(X,axis=0) - np.min(X,axis=0)+ 0.00000001)
#X_t = (X_t - np.min(X_t,axis=0))/ ( np.max(X_t,axis=0) - np.min(X_t,axis=0)+ 0.000000001)

pca_x = PCA(n_components=512)
pca_x.fit(X)

X = pca_x.transform(X)
X_t = pca_x.transform(X_t)

pca_y = PCA(n_components=2048)
pca_y.fit(y)

y = pca_y.transform(y)
y_t = pca_y.transform(y_t)

model = train_model(X,y)
test_model(model,X_t,y_t)


In [16]:
the_train_features = {}
the_test_features = {}

for fold in [1,2,3,4]:
    the_train_features[fold] = np.concatenate([train_features[fold]['inside_qout_feature'],
                                  train_features[fold]['start_sentence_feature'],
                                  train_features[fold]['end_sentence_feature'],
                                  train_features[fold]['pos_tag'],
                                  train_features[fold]['speach_state'],
                                  train_features[fold]['character']],axis=1)
    print(the_train_features[fold].shape)


    the_test_features[fold] = np.concatenate([test_features[fold]['inside_qout_feature'],
                                  test_features[fold]['start_sentence_feature'],
                                  test_features[fold]['end_sentence_feature'],
                                  test_features[fold]['pos_tag'],
                                  test_features[fold]['speach_state'],
                                  test_features[fold]['character']],axis=1)
    print(the_test_features[fold].shape)


(929, 168)
(366, 168)
(969, 168)
(326, 168)
(957, 168)
(338, 168)
(1030, 168)
(265, 168)


In [23]:
feat_acc = {}
feat_mrr = {}
for fold in [1,2,3,4]:
    feat_acc[fold] = []
    feat_mrr[fold]= []
    for delay in [1,2,3,4,5]:
        print(np.asarray(the_train_features[fold]).shape)
        X =  the_train_features[fold][delay:]## X usually means our input variables (or independent variables)
        y = np.asarray(train_brain_activations[fold])[:len(train_brain_activations[fold]) - delay] ## Y usually means our output/dependent variable

        X_t = the_test_features[fold][delay:] ## X usually means our input variables (or independent variables)
        y_t = np.asarray(test_brain_activations[fold])[:len(train_brain_activations[fold]) - delay] ## Y usually means our output/dependent variable

        pca_y = PCA(n_components=1024)
        pca_y.fit(y)

        y = pca_y.transform(y)
        y_t = pca_y.transform(y_t)

        model = train_model_regression(X,y)
        c_acc, e_acc, mean_ranks = test_model(model,X_t,y_t)
        feat_acc[fold].append(c_acc)
        feat_mrr[fold].append(mean_ranks) 


(929, 168)


/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:50: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


check argmax dim: (365,)
argmax accuracy: 0.0
cosine dist >>
MRR:  0.020267344723633677   187.1205479452055
euc_dists dist >>
MRR:  0.01992454262601652   186.5917808219178
binary accuracy >>
0.4787746500075267 0.4771789853981635
(929, 168)
check argmax dim: (364,)
argmax accuracy: 0.0
cosine dist >>
MRR:  0.012641590165573363   183.90934065934067
euc_dists dist >>
MRR:  0.018850531269609686   182.65384615384616
binary accuracy >>
0.489949444494899 0.4912209003118094
(929, 168)
check argmax dim: (363,)
argmax accuracy: 0.0
cosine dist >>
MRR:  0.023356724005072257   179.21763085399448
euc_dists dist >>
MRR:  0.020821764774084855   179.1404958677686
binary accuracy >>
0.513629514634035 0.5176780360105322
(929, 168)
check argmax dim: (362,)
argmax accuracy: 0.0
cosine dist >>
MRR:  0.018575165042383098   177.86187845303868
euc_dists dist >>
MRR:  0.016659244362901317   177.74861878453038
binary accuracy >>
0.5193523209011187 0.5157864128189039
(929, 168)
check argmax dim: (361,)
argmax ac

In [24]:
print(feat_acc)
print(mean_ranks)

{1: [0.4787746500075267, 0.489949444494899, 0.513629514634035, 0.5193523209011187, 0.5327331486611265], 2: [0.4641975308641975, 0.47593930359668235, 0.4909139857315924, 0.48888372903001104, 0.5195872274143302], 3: [0.49634378974141585, 0.5176439232409382, 0.5174010188578068, 0.5396774019528511, 0.505119577408734], 4: [0.4452413872565964, 0.4634139262183264, 0.4963294434207832, 0.5045977011494253, 0.5151173151173151]}
0.02274360394059322


Predicting PoS from the brain:

In [30]:
MRR_score['pos'] = {}
for fold in [1,2,3,4]:
    MRR_score['pos'][fold]= {}
    for ith_word in [0,1,2,3]:
        MRR_score['pos'][fold][ith_word] = []
        for delay in [0,1,2,3,4,5]:        
            print("Delay:",delay,"ith word:",ith_word)
            y =  np.asarray(train_features[fold]['pos_tag'])[delay:,ith_word*28: (ith_word+1)*28]## X usually means our input variables (or independent variables)
            X = np.asarray(train_brain_activations[fold])[:len(train_brain_activations[fold])-delay] ## Y usually means our output/dependent variable

            y_t = np.asarray(test_features[fold]['pos_tag'])[delay:,ith_word*28: (ith_word+1)*28] ## X usually means our input variables (or independent variables)
            X_t = np.asarray(test_brain_activations[fold])[:len(test_brain_activations[fold])-delay] ## Y usually means our output/dependent variable



            y = np.min([y,np.ones(y.shape)],axis=0)
            y_t = np.min([y_t,np.ones(y_t.shape)],axis=0)  
            y = np.argmax(y,axis=1)
            y_t = np.argmax(y_t,axis=1)  


            pca_x = PCA(n_components=512)
            pca_x.fit(X)

            X = pca_x.transform(X)
            X_t = pca_x.transform(X_t)


            clf =SVC(kernel='linear',class_weight='balanced', C=0.1, probability=True)
            clf.fit(X, y)

            prediction_train = clf.predict(X)
            prediction_test = clf.predict(X_t)


            prediction_train_prob = clf.predict_proba(X)
            prediction_test_prob = clf.predict_proba(X_t)

            train_accuracy = np.mean(prediction_train == y)
            test_accuracy = np.mean(prediction_test == y_t)

            print("train accuracy: ",train_accuracy, " test accuracy: ",test_accuracy)
            cnf_matrix = confusion_matrix(y_t, prediction_test)
            cnf_matrix = confusion_matrix(y, prediction_train)

            result = []
            for i in np.arange(len(X_t)):
                j = np.where (np.argsort(prediction_test_prob[i]) == y_t[i])[0]
                if len(j) == 0:
                    result.append(0)
                else:
                    result.append(1/((np.max(y_t)+1) - j[0]))

            print("MRR: ",np.mean(result))
            MRR_score['pos'][fold][ith_word].append(np.mean(result))

Delay: 0 ith word: 0
train accuracy:  1.0  test accuracy:  0.09016393442622951
MRR:  0.1378385308015019
Delay: 1 ith word: 0
train accuracy:  1.0  test accuracy:  0.06301369863013699
MRR:  0.1389728483949751
Delay: 2 ith word: 0
train accuracy:  1.0  test accuracy:  0.09065934065934066
MRR:  0.13673261200150644
Delay: 3 ith word: 0
train accuracy:  1.0  test accuracy:  0.08539944903581267
MRR:  0.13799556745137628
Delay: 4 ith word: 0
train accuracy:  1.0  test accuracy:  0.10497237569060773
MRR:  0.1384945413587851
Delay: 5 ith word: 0
train accuracy:  1.0  test accuracy:  0.10249307479224377
MRR:  0.1389899113153623
Delay: 0 ith word: 1
train accuracy:  1.0  test accuracy:  0.08196721311475409
MRR:  0.17749062852276462
Delay: 1 ith word: 1
train accuracy:  1.0  test accuracy:  0.07945205479452055
MRR:  0.17728312641934382
Delay: 2 ith word: 1
train accuracy:  1.0  test accuracy:  0.07967032967032966
MRR:  0.17678191253391706
Delay: 3 ith word: 1
train accuracy:  1.0  test accuracy:  

train accuracy:  1.0  test accuracy:  0.11363636363636363
MRR:  0.1273347301308656
Delay: 2 ith word: 1
train accuracy:  1.0  test accuracy:  0.09885931558935361
MRR:  0.12855893427032913
Delay: 3 ith word: 1
train accuracy:  1.0  test accuracy:  0.11450381679389313
MRR:  0.12445261840163602
Delay: 4 ith word: 1
train accuracy:  1.0  test accuracy:  0.11877394636015326
MRR:  0.12550658913358753
Delay: 5 ith word: 1
train accuracy:  1.0  test accuracy:  0.12692307692307692
MRR:  0.12712576048021468
Delay: 0 ith word: 2
train accuracy:  1.0  test accuracy:  0.07547169811320754
MRR:  0.30842755643471215
Delay: 1 ith word: 2
train accuracy:  1.0  test accuracy:  0.10606060606060606
MRR:  0.309614739923194
Delay: 2 ith word: 2
train accuracy:  1.0  test accuracy:  0.09505703422053231
MRR:  0.3123535629962696
Delay: 3 ith word: 2
train accuracy:  1.0  test accuracy:  0.09923664122137404
MRR:  0.31226953309383265
Delay: 4 ith word: 2
train accuracy:  1.0  test accuracy:  0.0842911877394636
MR

In [53]:
for d in [0,1,2,3,4,5]:
    for i in [0,1,2,3]:
        mrr = []
        for fold in [1,2,3,4]:
            mrr.append(MRR_score['pos'][fold][i][d])
        print(i,d,np.mean(mrr))

0 0 0.13465190411077904
1 0 0.1649560555005874
2 0 0.22956995216244086
3 0 0.20674196063257877
0 1 0.13407731382557997
1 1 0.16625200457602188
2 1 0.22816647269637944
3 1 0.20568435682446254
0 2 0.13405348331934303
1 2 0.16456763244374376
2 2 0.2294665588767334
3 2 0.20755803373420578
0 3 0.1350476466324249
1 3 0.16351000636884697
2 3 0.2308837268476082
3 3 0.20703990005560405
0 4 0.1338440656090332
1 4 0.16312691895941978
2 4 0.22891598320148948
3 4 0.20706520782618693
0 5 0.13479981098981614
1 5 0.16222314092194434
2 5 0.23093708169185145
3 5 0.20678692828483086


In [41]:
MRR_score['character'] = {}
for fold in [1,2,3,4]:
    MRR_score['character'][fold] = {}
    for ith_word in [0,1,2,3]:
        MRR_score['character'][fold][ith_word] = []
        for delay in [0,1,2,3,4,5]:        
            print("Delay:",delay,"ith word:",ith_word)
            y =  np.asarray(train_features[fold]['character'])[delay:,ith_word*10: (ith_word+1)*10]## X usually means our input variables (or independent variables)
            X = np.asarray(train_brain_activations[fold])[:len(train_brain_activations[fold])-delay] ## Y usually means our output/dependent variable

            y_t = np.asarray(test_features[fold]['character'])[delay:,ith_word*10: (ith_word+1)*10] ## X usually means our input variables (or independent variables)
            X_t = np.asarray(test_brain_activations[fold])[:len(test_brain_activations[fold])-delay] ## Y usually means our output/dependent variable


            print(y.shape)
            print(X.shape)

            y = np.min([y,np.ones(y.shape)],axis=0)
            y_t = np.min([y_t,np.ones(y_t.shape)],axis=0)  


            y_new = np.argmax(y,axis=1)
            y_t_new = np.argmax(y_t,axis=1)  

            y_new[np.where(np.max(y,axis=1) > 0)] +=1
            y_t_new[np.where(np.max(y_t,axis=1) > 0)] +=1

            y_new[np.where(np.max(y,axis=1) == 0)] = 0 
            y_t_new[np.where(np.max(y_t,axis=1) == 0)] = 0

            y = y_new
            y_t = y_t_new
            pca_x = PCA(n_components=512)
            pca_x.fit(X)

            X = pca_x.transform(X)
            X_t = pca_x.transform(X_t)


            clf =SVC(kernel='linear',class_weight='balanced', C=0.1, probability=True)
            clf.fit(X, y)

            prediction_train = clf.predict(X)
            prediction_test = clf.predict(X_t)


            prediction_train_prob = clf.predict_proba(X)
            prediction_test_prob = clf.predict_proba(X_t)

            train_accuracy = np.mean(prediction_train == y)
            test_accuracy = np.mean(prediction_test == y_t)

            print("train accuracy: ",train_accuracy, " test accuracy: ",test_accuracy)
            cnf_matrix = confusion_matrix(y_t, prediction_test)
            cnf_matrix = confusion_matrix(y, prediction_train)

            result = []
            result_accu = []
            for i in np.arange(len(X_t)):
                if (y_t[i] > 0):
                    j = np.where (np.argsort(prediction_test_prob[i]) == y_t[i])[0]
                    if len(j) == 0:
                        result.append(0)
                        result_accu.append(0)
                    else:
                        result.append(1/((np.max(y_t)+1) - j[0]))
                        result_accu.append(j[0] == 0)
            MRR_score['character'][fold][ith_word].append(np.mean(result))
            print("MRR: ",np.mean(result),"important accuracy:",np.mean(result_accu))

Delay: 0 ith word: 0
(929, 10)
(929, 37913)
train accuracy:  1.0  test accuracy:  0.9016393442622951
MRR:  0.17050264550264546 important accuracy: 0.08333333333333333
Delay: 1 ith word: 0
(928, 10)
(928, 37913)
train accuracy:  1.0  test accuracy:  0.9041095890410958
MRR:  0.1743310657596372 important accuracy: 0.08571428571428572
Delay: 2 ith word: 0
(927, 10)
(927, 37913)
train accuracy:  1.0  test accuracy:  0.8956043956043956
MRR:  0.17060090702947844 important accuracy: 0.11428571428571428
Delay: 3 ith word: 0
(926, 10)
(926, 37913)
train accuracy:  1.0  test accuracy:  0.9035812672176309
MRR:  0.17866213151927435 important accuracy: 0.14285714285714285
Delay: 4 ith word: 0
(925, 10)
(925, 37913)
train accuracy:  1.0  test accuracy:  0.9060773480662984
MRR:  0.17644724556489264 important accuracy: 0.08823529411764706
Delay: 5 ith word: 0
(924, 10)
(924, 37913)
train accuracy:  1.0  test accuracy:  0.9058171745152355
MRR:  0.1853874883286648 important accuracy: 0.058823529411764705

train accuracy:  1.0  test accuracy:  0.8452380952380952
MRR:  0.1897664835164835 important accuracy: 0.21153846153846154
Delay: 3 ith word: 0
(954, 10)
(954, 37913)
train accuracy:  1.0  test accuracy:  0.844776119402985
MRR:  0.17157217782217782 important accuracy: 0.17307692307692307
Delay: 4 ith word: 0
(953, 10)
(953, 37913)
train accuracy:  1.0  test accuracy:  0.844311377245509
MRR:  0.19350441225441228 important accuracy: 0.17307692307692307
Delay: 5 ith word: 0
(952, 10)
(952, 37913)
train accuracy:  1.0  test accuracy:  0.8468468468468469
MRR:  0.18540234275528392 important accuracy: 0.19607843137254902
Delay: 0 ith word: 1
(957, 10)
(957, 37913)
train accuracy:  1.0  test accuracy:  0.8402366863905325
MRR:  0.18252498530276307 important accuracy: 0.0
Delay: 1 ith word: 1
(956, 10)
(956, 37913)
train accuracy:  1.0  test accuracy:  0.8397626112759644
MRR:  0.17286155202821868 important accuracy: 0.0
Delay: 2 ith word: 1
(955, 10)
(955, 37913)
train accuracy:  1.0  test accura

In [50]:
for d in [0,1,2,3,4,5]:
    for i in [0,1,2,3]:
        mrr = []
        for fold in [1,2,3,4]:
            mrr.append(MRR_score['character'][fold][i][d])
        print(i,d,np.mean(mrr))

0 0 0.32035221536229597
1 0 0.3259648056915082
2 0 0.32463809181225023
3 0 0.3208271892490753
0 1 0.3252053283058467
1 1 0.32282672697659254
2 1 0.32952433895013944
3 1 0.3855541727652809
0 2 0.3155859969091027
1 2 0.3245148969833237
2 2 0.33130427027132686
3 2 0.38812058281009676
0 3 0.32574643312283935
1 3 0.3267242174789345
2 3 0.32864522245679884
3 3 0.3941963246495783
0 4 0.3266066156987647
1 4 0.3327163672699387
2 4 0.3259384285323324
3 4 0.3892329400685111
0 5 0.3135387916163533
1 5 0.32832427354748783
2 5 0.3341663072984912
3 5 0.38999209774160537


In [49]:
Accuracy_score['inside_qout_feature'] = {}
for fold in [1,2,3,4]:
    Accuracy_score['inside_qout_feature'][fold]={}
    for ith_word in [0,1,2,3]:
        Accuracy_score['inside_qout_feature'][fold][ith_word]=[]
        for delay in [0,1,2,3,4,5]:        
            print("Delay:",delay,"ith word:",ith_word)
            y =  np.asarray(train_features[fold]['inside_qout_feature'])[delay:,ith_word]## X usually means our input variables (or independent variables)
            X = np.asarray(train_brain_activations[fold])[:len(train_brain_activations[fold])-delay] ## Y usually means our output/dependent variable

            y_t = np.asarray(test_features[fold]['inside_qout_feature'])[delay:,ith_word] ## X usually means our input variables (or independent variables)
            X_t = np.asarray(test_brain_activations[fold])[:len(test_brain_activations[fold])-delay] ## Y usually means our output/dependent variable


            print(y.shape)


            pca_x = PCA(n_components=512)
            pca_x.fit(X)

            X = pca_x.transform(X)
            X_t = pca_x.transform(X_t)


            clf =SVC(kernel='linear',class_weight='balanced', C=0.1, probability=True)
            clf.fit(X, y)

            prediction_train = clf.predict(X)
            prediction_test = clf.predict(X_t)


            prediction_train_prob = clf.predict_proba(X)
            prediction_test_prob = clf.predict_proba(X_t)

            train_accuracy = np.mean(prediction_train == y)
            test_accuracy = np.mean(prediction_test == y_t)

            print("train accuracy: ",train_accuracy, " test accuracy: ",test_accuracy)
            cnf_matrix = confusion_matrix(y_t, prediction_test)
            cnf_matrix = confusion_matrix(y, prediction_train)

            result_accu = []
            result = []
            for i in np.arange(len(X_t)):
                if (y_t[i] == 1):
                    result_accu.append(prediction_test[i] == 1)

                    if prediction_test[i] == 0:
                        result.append(1/2)
                    else:
                        result.append(1) 

            Accuracy_score['inside_qout_feature'][fold][ith_word].append(np.mean(result_accu))
            print("MRR: ",np.mean(result),"important accuracy:",np.mean(result_accu))

Delay: 0 ith word: 0
(929,)
train accuracy:  1.0  test accuracy:  0.6366120218579235
MRR:  0.7043010752688172 important accuracy: 0.40860215053763443
Delay: 1 ith word: 0
(928,)
train accuracy:  1.0  test accuracy:  0.6219178082191781
MRR:  0.7204301075268817 important accuracy: 0.44086021505376344
Delay: 2 ith word: 0
(927,)
train accuracy:  0.9967637540453075  test accuracy:  0.5824175824175825
MRR:  0.6827956989247311 important accuracy: 0.3655913978494624
Delay: 3 ith word: 0
(926,)
train accuracy:  0.9989200863930886  test accuracy:  0.5702479338842975
MRR:  0.6881720430107527 important accuracy: 0.3763440860215054
Delay: 4 ith word: 0
(925,)
train accuracy:  0.987027027027027  test accuracy:  0.5220994475138122
MRR:  0.7311827956989247 important accuracy: 0.46236559139784944
Delay: 5 ith word: 0
(924,)
train accuracy:  0.9783549783549783  test accuracy:  0.556786703601108
MRR:  0.7445652173913043 important accuracy: 0.4891304347826087
Delay: 0 ith word: 1
(929,)
train accuracy:  

train accuracy:  1.0  test accuracy:  0.6331360946745562
MRR:  0.6081081081081081 important accuracy: 0.21621621621621623
Delay: 1 ith word: 1
(956,)
train accuracy:  0.997907949790795  test accuracy:  0.629080118694362
MRR:  0.6681818181818182 important accuracy: 0.33636363636363636
Delay: 2 ith word: 1
(955,)
train accuracy:  1.0  test accuracy:  0.6160714285714286
MRR:  0.6363636363636364 important accuracy: 0.2727272727272727
Delay: 3 ith word: 1
(954,)
train accuracy:  0.980083857442348  test accuracy:  0.5791044776119403
MRR:  0.65 important accuracy: 0.3
Delay: 4 ith word: 1
(953,)
train accuracy:  0.9926547743966422  test accuracy:  0.6077844311377245
MRR:  0.6545454545454545 important accuracy: 0.3090909090909091
Delay: 5 ith word: 1
(952,)
train accuracy:  0.9600840336134454  test accuracy:  0.5375375375375375
MRR:  0.6513761467889908 important accuracy: 0.30275229357798167
Delay: 0 ith word: 2
(957,)
train accuracy:  1.0  test accuracy:  0.5857988165680473
MRR:  0.6081081081

In [51]:
for d in [0,1,2,3,4,5]:
    for i in [0,1,2,3]:
        mrr = []
        for fold in [1,2,3,4]:
            mrr.append(Accuracy_score['inside_qout_feature'][fold][i][d])
        print(i,d,np.mean(mrr))


0 0 0.2916432766961132
1 0 0.2802799610835325
2 0 0.238016761000632
3 0 0.24901294749213618
0 1 0.31840251847264467
1 1 0.313604797979798
2 1 0.2728450014740337
3 1 0.2681888178156835
0 2 0.2563949296768371
1 2 0.29648633443276295
2 2 0.2626892969634905
3 2 0.2734086265103056
0 3 0.24949594974139155
1 3 0.28247590702947845
2 3 0.30320582941550683
3 3 0.30496426597172865
0 4 0.2980714781556296
1 4 0.2937173606816464
2 4 0.25783873936047846
3 4 0.2150552973436909
0 5 0.3082735183707724
1 5 0.27170396686528575
2 5 0.2947087146628431
3 5 0.23745047047682805


In [52]:
Accuracy_score['speach_state'] = {}
for fold in [1,2,3,4]:
    Accuracy_score['speach_state'][fold]={}
    for ith_word in [0,1,2,3]:
        Accuracy_score['speach_state'][fold][ith_word]=[]
        for delay in [0,1,2,3,4,5]:        
            print("Delay:",delay,"ith word:",ith_word)
            y =  np.asarray(train_features[fold]['speach_state'])[delay:,ith_word]## X usually means our input variables (or independent variables)
            X = np.asarray(train_brain_activations[fold])[:len(train_brain_activations[fold])-delay] ## Y usually means our output/dependent variable

            y_t = np.asarray(test_features[fold]['speach_state'])[delay:,ith_word] ## X usually means our input variables (or independent variables)
            X_t = np.asarray(test_brain_activations[fold])[:len(test_brain_activations[fold])-delay] ## Y usually means our output/dependent variable


            print(y.shape)


            pca_x = PCA(n_components=512)
            pca_x.fit(X)

            X = pca_x.transform(X)
            X_t = pca_x.transform(X_t)


            clf =SVC(kernel='linear',class_weight='balanced', C=0.1, probability=True)
            clf.fit(X, y)

            prediction_train = clf.predict(X)
            prediction_test = clf.predict(X_t)


            prediction_train_prob = clf.predict_proba(X)
            prediction_test_prob = clf.predict_proba(X_t)

            train_accuracy = np.mean(prediction_train == y)
            test_accuracy = np.mean(prediction_test == y_t)

            print("train accuracy: ",train_accuracy, " test accuracy: ",test_accuracy)
            cnf_matrix = confusion_matrix(y_t, prediction_test)
            cnf_matrix = confusion_matrix(y, prediction_train)

            result = []
            result_accu = []
            for i in np.arange(len(X_t)):
                if (y_t[i] == 1):
                    result_accu.append(prediction_test[i] == 1)

                    if prediction_test[i] == 0:
                        result.append(1/2)
                    else:
                        result.append(1) 
            Accuracy_score['speach_state'][fold][ith_word].append(np.mean(result_accu))
            print("MRR: ",np.mean(result),"important accuracy:",np.mean(result_accu))

Delay: 0 ith word: 0
(929,)
train accuracy:  1.0  test accuracy:  0.9590163934426229
MRR:  0.5 important accuracy: 0.0
Delay: 1 ith word: 0
(928,)
train accuracy:  1.0  test accuracy:  0.958904109589041
MRR:  0.5 important accuracy: 0.0
Delay: 2 ith word: 0
(927,)
train accuracy:  1.0  test accuracy:  0.9587912087912088
MRR:  0.5 important accuracy: 0.0
Delay: 3 ith word: 0
(926,)
train accuracy:  1.0  test accuracy:  0.9586776859504132
MRR:  0.5 important accuracy: 0.0
Delay: 4 ith word: 0
(925,)
train accuracy:  1.0  test accuracy:  0.9585635359116023
MRR:  0.5 important accuracy: 0.0
Delay: 5 ith word: 0
(924,)
train accuracy:  1.0  test accuracy:  0.9584487534626038
MRR:  0.5 important accuracy: 0.0
Delay: 0 ith word: 1
(929,)
train accuracy:  1.0  test accuracy:  0.953551912568306
MRR:  0.5 important accuracy: 0.0
Delay: 1 ith word: 1
(928,)
train accuracy:  1.0  test accuracy:  0.9534246575342465
MRR:  0.5 important accuracy: 0.0
Delay: 2 ith word: 1
(927,)
train accuracy:  1.0  

train accuracy:  1.0  test accuracy:  0.9462686567164179
MRR:  0.5 important accuracy: 0.0
Delay: 4 ith word: 3
(953,)
train accuracy:  1.0  test accuracy:  0.9461077844311377
MRR:  0.5 important accuracy: 0.0
Delay: 5 ith word: 3
(952,)
train accuracy:  1.0  test accuracy:  0.9459459459459459
MRR:  0.5 important accuracy: 0.0
Delay: 0 ith word: 0
(1030,)
train accuracy:  1.0  test accuracy:  0.9622641509433962
MRR:  0.5 important accuracy: 0.0
Delay: 1 ith word: 0
(1029,)
train accuracy:  1.0  test accuracy:  0.9621212121212122
MRR:  0.5 important accuracy: 0.0
Delay: 2 ith word: 0
(1028,)
train accuracy:  1.0  test accuracy:  0.9619771863117871
MRR:  0.5 important accuracy: 0.0
Delay: 3 ith word: 0
(1027,)
train accuracy:  1.0  test accuracy:  0.9618320610687023
MRR:  0.5 important accuracy: 0.0
Delay: 4 ith word: 0
(1026,)
train accuracy:  1.0  test accuracy:  0.9616858237547893
MRR:  0.5 important accuracy: 0.0
Delay: 5 ith word: 0
(1025,)
train accuracy:  1.0  test accuracy:  0.96

In [47]:
Accuracy_score['start_sentence_feature'] = {}
for fold in [1,2,3,4]:
    Accuracy_score['start_sentence_feature'][fold]={}
    for ith_word in [0,1,2,3]:
        Accuracy_score['start_sentence_feature'][fold][ith_word]=[]
        for delay in [0,1,2,3,4,5]:        
            print("Delay:",delay,"ith word:",ith_word)
            y =  np.asarray(train_features[fold]['start_sentence_feature'])[delay:,ith_word]## X usually means our input variables (or independent variables)
            X = np.asarray(train_brain_activations[fold])[:len(train_brain_activations[fold])-delay] ## Y usually means our output/dependent variable

            y_t = np.asarray(test_features[fold]['start_sentence_feature'])[delay:,ith_word] ## X usually means our input variables (or independent variables)
            X_t = np.asarray(test_brain_activations[fold])[:len(test_brain_activations[fold])-delay] ## Y usually means our output/dependent variable


            print(y.shape)


            pca_x = PCA(n_components=512)
            pca_x.fit(X)

            X = pca_x.transform(X)
            X_t = pca_x.transform(X_t)


            clf =SVC(kernel='linear',class_weight='balanced', C=0.1, probability=True)
            clf.fit(X, y)

            prediction_train = clf.predict(X)
            prediction_test = clf.predict(X_t)


            prediction_train_prob = clf.predict_proba(X)
            prediction_test_prob = clf.predict_proba(X_t)

            train_accuracy = np.mean(prediction_train == y)
            test_accuracy = np.mean(prediction_test == y_t)

            print("train accuracy: ",train_accuracy, " test accuracy: ",test_accuracy)
            cnf_matrix = confusion_matrix(y_t, prediction_test)
            cnf_matrix = confusion_matrix(y, prediction_train)

            result = []
            result_accu = []
            for i in np.arange(len(X_t)):
                if (y_t[i] == 1):
                    result_accu.append(prediction_test[i] == 1)

                    if prediction_test[i] == 0:
                        result.append(1/2)
                    else:
                        result.append(1) 

            Accuracy_score['start_sentence_feature'][fold][ith_word].append(np.mean(result_accu))
            print("MRR: ",np.mean(result),"important accuracy:",np.mean(result_accu))

Delay: 0 ith word: 0
(929,)
train accuracy:  1.0  test accuracy:  0.8989071038251366
MRR:  0.5 important accuracy: 0.0
Delay: 1 ith word: 0
(928,)
train accuracy:  1.0  test accuracy:  0.9068493150684932
MRR:  0.5 important accuracy: 0.0
Delay: 2 ith word: 0
(927,)
train accuracy:  1.0  test accuracy:  0.9065934065934066
MRR:  0.5 important accuracy: 0.0
Delay: 3 ith word: 0
(926,)
train accuracy:  1.0  test accuracy:  0.9035812672176309
MRR:  0.5 important accuracy: 0.0
Delay: 4 ith word: 0
(925,)
train accuracy:  1.0  test accuracy:  0.9088397790055248
MRR:  0.5147058823529411 important accuracy: 0.029411764705882353
Delay: 5 ith word: 0
(924,)
train accuracy:  1.0  test accuracy:  0.9002770083102493
MRR:  0.5 important accuracy: 0.0
Delay: 0 ith word: 1
(929,)
train accuracy:  1.0  test accuracy:  0.9098360655737705
MRR:  0.5 important accuracy: 0.0
Delay: 1 ith word: 1
(928,)
train accuracy:  1.0  test accuracy:  0.9123287671232877
MRR:  0.5 important accuracy: 0.0
Delay: 2 ith wor

train accuracy:  1.0  test accuracy:  0.9226190476190477
MRR:  0.5 important accuracy: 0.0
Delay: 3 ith word: 3
(954,)
train accuracy:  1.0  test accuracy:  0.9223880597014925
MRR:  0.5 important accuracy: 0.0
Delay: 4 ith word: 3
(953,)
train accuracy:  1.0  test accuracy:  0.9221556886227545
MRR:  0.5 important accuracy: 0.0
Delay: 5 ith word: 3
(952,)
train accuracy:  1.0  test accuracy:  0.9219219219219219
MRR:  0.5 important accuracy: 0.0
Delay: 0 ith word: 0
(1030,)
train accuracy:  1.0  test accuracy:  0.8754716981132076
MRR:  0.5 important accuracy: 0.0
Delay: 1 ith word: 0
(1029,)
train accuracy:  1.0  test accuracy:  0.8939393939393939
MRR:  0.5 important accuracy: 0.0
Delay: 2 ith word: 0
(1028,)
train accuracy:  1.0  test accuracy:  0.8897338403041825
MRR:  0.5 important accuracy: 0.0
Delay: 3 ith word: 0
(1027,)
train accuracy:  1.0  test accuracy:  0.8969465648854962
MRR:  0.5 important accuracy: 0.0
Delay: 4 ith word: 0
(1026,)
train accuracy:  1.0  test accuracy:  0.896

In [48]:
Accuracy_score['end_sentence_feature'] = {}
for fold in [1,2,3,4]:
    Accuracy_score['end_sentence_feature'][fold]={}
    for ith_word in [0,1,2,3]:
        Accuracy_score['end_sentence_feature'][fold][ith_word]=[]
        for delay in [0,1,2,3,4,5]:        
            print("Delay:",delay,"ith word:",ith_word)
            y =  np.asarray(train_features[fold]['end_sentence_feature'])[delay:,ith_word]## X usually means our input variables (or independent variables)
            X = np.asarray(train_brain_activations[fold])[:len(train_brain_activations[fold])-delay] ## Y usually means our output/dependent variable

            y_t = np.asarray(test_features[fold]['end_sentence_feature'])[delay:,ith_word] ## X usually means our input variables (or independent variables)
            X_t = np.asarray(test_brain_activations[fold])[:len(test_brain_activations[fold])-delay] ## Y usually means our output/dependent variable


            print(y.shape)


            pca_x = PCA(n_components=512)
            pca_x.fit(X)

            X = pca_x.transform(X)
            X_t = pca_x.transform(X_t)


            clf =SVC(kernel='linear',class_weight='balanced', C=0.1, probability=True)
            clf.fit(X, y)

            prediction_train = clf.predict(X)
            prediction_test = clf.predict(X_t)


            prediction_train_prob = clf.predict_proba(X)
            prediction_test_prob = clf.predict_proba(X_t)

            train_accuracy = np.mean(prediction_train == y)
            test_accuracy = np.mean(prediction_test == y_t)

            print("train accuracy: ",train_accuracy, " test accuracy: ",test_accuracy)
            cnf_matrix = confusion_matrix(y_t, prediction_test)
            cnf_matrix = confusion_matrix(y, prediction_train)

            result = []
            result_accu = []
            for i in np.arange(len(X_t)):
                if (y_t[i] == 1):
                    result_accu.append(prediction_test[i] == 1)

                    if prediction_test[i] == 0:
                        result.append(1/2)
                    else:
                        result.append(1) 
            
            Accuracy_score['end_sentence_feature'][fold][ith_word].append(np.mean(result_accu))
            print("MRR: ",np.mean(result),"important accuracy:",np.mean(result_accu))

Delay: 0 ith word: 0
(929,)
train accuracy:  1.0  test accuracy:  0.912568306010929
MRR:  0.5 important accuracy: 0.0
Delay: 1 ith word: 0
(928,)
train accuracy:  1.0  test accuracy:  0.9123287671232877
MRR:  0.5 important accuracy: 0.0
Delay: 2 ith word: 0
(927,)
train accuracy:  1.0  test accuracy:  0.9148351648351648
MRR:  0.515625 important accuracy: 0.03125
Delay: 3 ith word: 0
(926,)
train accuracy:  1.0  test accuracy:  0.9090909090909091
MRR:  0.5 important accuracy: 0.0
Delay: 4 ith word: 0
(925,)
train accuracy:  1.0  test accuracy:  0.914364640883978
MRR:  0.5 important accuracy: 0.0
Delay: 5 ith word: 0
(924,)
train accuracy:  1.0  test accuracy:  0.9141274238227147
MRR:  0.5 important accuracy: 0.0
Delay: 0 ith word: 1
(929,)
train accuracy:  1.0  test accuracy:  0.8934426229508197
MRR:  0.5 important accuracy: 0.0
Delay: 1 ith word: 1
(928,)
train accuracy:  1.0  test accuracy:  0.8958904109589041
MRR:  0.5 important accuracy: 0.0
Delay: 2 ith word: 1
(927,)
train accurac

train accuracy:  1.0  test accuracy:  0.9226190476190477
MRR:  0.5 important accuracy: 0.0
Delay: 3 ith word: 3
(954,)
train accuracy:  1.0  test accuracy:  0.9253731343283582
MRR:  0.5 important accuracy: 0.0
Delay: 4 ith word: 3
(953,)
train accuracy:  1.0  test accuracy:  0.9311377245508982
MRR:  0.5 important accuracy: 0.0
Delay: 5 ith word: 3
(952,)
train accuracy:  1.0  test accuracy:  0.9309309309309309
MRR:  0.5 important accuracy: 0.0
Delay: 0 ith word: 0
(1030,)
train accuracy:  1.0  test accuracy:  0.9207547169811321
MRR:  0.5 important accuracy: 0.0
Delay: 1 ith word: 0
(1029,)
train accuracy:  1.0  test accuracy:  0.9204545454545454
MRR:  0.5 important accuracy: 0.0
Delay: 2 ith word: 0
(1028,)
train accuracy:  1.0  test accuracy:  0.9239543726235742
MRR:  0.525 important accuracy: 0.05
Delay: 3 ith word: 0
(1027,)
train accuracy:  1.0  test accuracy:  0.9198473282442748
MRR:  0.5 important accuracy: 0.0
Delay: 4 ith word: 0
(1026,)
train accuracy:  1.0  test accuracy:  0.

In [64]:
Accuracy_score['end_sentence_feature'] = {}
for fold in [1,2,3,4]:
    print(np.asarray(train_features[fold]['step']).shape)
    X =  train_features[fold]['step']## X usually means our input variables (or independent variables)
    y = np.asarray(train_brain_activations[fold]) ## Y usually means our output/dependent variable

    X_t = test_features[fold]['step'] ## X usually means our input variables (or independent variables)
    y_t = np.asarray(test_brain_activations[fold]) ## Y usually means our output/dependent variable


    #X = X - np.mean(X,axis=0)
    #X_t = X_t - np.mean(X_t, axis=0)
    y = y - np.mean(y,axis=0)
    y_t = y_t - np.mean(y_t, axis=0)

    #y = (y - np.min(y,axis=0))/ ( np.max(y,axis=0) - np.min(y,axis=0)+ 0.0000001)
    #y_t = (y_t - np.min(y_t,axis=0))/ ( np.max(y_t,axis=0) - np.min(y_t,axis=0)+ 0.0000001)

    #X = (X - np.min(X,axis=0))/ ( np.max(X,axis=0) - np.min(X,axis=0)+ 0.00000001)
    #X_t = (X_t - np.min(X_t,axis=0))/ ( np.max(X_t,axis=0) - np.min(X_t,axis=0)+ 0.000000001)

    #pca_x = PCA(n_components=512)
    #pca_x.fit(X)

    #X = pca_x.transform(X)
    #X_t = pca_x.transform(X_t)

    pca_y = PCA(n_components=512)
    pca_y.fit(y)

    y = pca_y.transform(y)
    y_t = pca_y.transform(y_t)

    model = train_model_regression(X,y)
    test_model(model,X_t,y_t)


(929, 4)
check argmax dim: (366,)
argmax accuracy: 0.00546448087431694
cosine dist >>
MRR:  0.01771603346755243   183.29234972677597
euc_dists dist >>
MRR:  0.018575493876285508   180.65573770491804
binary accuracy >>
0.5632457519275395 0.7610599595778127
(969, 4)
check argmax dim: (326,)
argmax accuracy: 0.046012269938650305
cosine dist >>
MRR:  0.019509038852099588   163.57055214723925
euc_dists dist >>
MRR:  0.020142087298729856   161.41411042944785
binary accuracy >>
0.39652666352052857 0.6267107126002831
(957, 4)
check argmax dim: (338,)
argmax accuracy: 0.005917159763313609
cosine dist >>
MRR:  0.018937246767278742   169.90828402366864
euc_dists dist >>
MRR:  0.01902584656523635   166.3639053254438
binary accuracy >>
0.43451618000807685 0.7192597404877706
(1030, 4)
check argmax dim: (265,)
argmax accuracy: 0.03018867924528302
cosine dist >>
MRR:  0.02326423326692522   132.81132075471697
euc_dists dist >>
MRR:  0.02404691320778303   127.20754716981132
binary accuracy >>
0.53001715